In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/ahmadelsallab/MultiCheXNet.git

In [ ]:
from MultiCheXNet.data_loader.MTL_dataloader import get_train_validation_generator

In [ ]:
det_csv_path = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_labels.csv"
seg_csv_path = "/kaggle/input/siim-acr-pneumothorax-segmentation-data/train-rle.csv"
det_images_path = "/kaggle/input/rsna-pneumonia-detection-challenge/stage_2_train_images/"
seg_images_path = "/kaggle/input/siim-acr-pneumothorax-segmentation-data/dicom-images-train/"

# Prepare data generators

In [ ]:
from MultiCheXNet.data_loader.SIIM_ACR_dataloader import get_train_validation_generator


seg_train_gen , seg_val_gen = get_train_validation_generator(seg_csv_path,seg_images_path)

# Build MTL model

In [ ]:
from MultiCheXNet.utils.ModelBlock import ModelBlock
from MultiCheXNet.utils.Encoder import Encoder
from MultiCheXNet.utils.Detector import Detector
from MultiCheXNet.utils.Segmenter import Segmenter
from MultiCheXNet.utils.Classifier import Classifier

import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np
from MultiCheXNet.utils.loss.MTL_loss import MTL_loss
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *

In [ ]:
class Segmenter(ModelBlock):
    def __init__(self, encoder):
        self.encoder = encoder.model
        self.model = self.make_model()
        #self.num_layers = ModelBlock.get_head_num_layers(encoder, self.model)

    def dense_block(self, x, blocks, name):
        #REF: keras-team
        """A dense block.
        # Arguments
            x: input tensor.
            blocks: integer, the number of building blocks.
            name: string, block label.
        # Returns
            output tensor for the block.
        """
        for i in range(blocks):
            x = self.conv_block(x, 32, name=name + '_block' + str(i + 1))
        return x

    def conv_block(self, x, growth_rate, name):
        #REF: keras-team
        """A building block for a dense block.
        # Arguments
            x: input tensor.
            growth_rate: float, growth rate at dense layers.
            name: string, block label.
        # Returns
            Output tensor for the block.
        """
        bn_axis = 3
        
        x1 = Dropout(0.3)(x)
        x1 = BatchNormalization(axis=bn_axis,
                                       epsilon=1.001e-5,
                                       name=name + '_0_bn')(x1)
        x1 = Activation('relu', name=name + '_0_relu')(x1)
        x1 = Conv2D(4 * growth_rate, 1,
                           use_bias=False,
                           name=name + '_1_conv')(x1)
        x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                       name=name + '_1_bn')(x1)
        x1 = Activation('relu', name=name + '_1_relu')(x1)
        x1 = Conv2D(growth_rate, 3,
                           padding='same',
                           use_bias=False,
                           name=name + '_2_conv')(x1)
        x = Concatenate(axis=bn_axis, name=name + '_concat')([x, x1])
        return x

    def transition_up(self, x, skip_connection, out_channels, kernel_size=(3,3), stride=(2,2)):
        tu = Conv2DTranspose(out_channels, kernel_size, strides = stride, padding = 'same')(x)
        skip = self.encoder.layers[skip_connection].output
        c = concatenate([skip,tu], axis=3)
        return c

    def make_model(self, skip_layers=[308, 136, 48], blocks=[3, 3, 3, 3]):
        """
        This model is responsible for building a keras model
        :return:
            keras model:
        """
        db5 = self.encoder.output #(8,8,1024)
        tu5 = self.transition_up(db5, skip_layers[0], 3)

        db6 = self.dense_block(tu5, blocks[-1], name='conv6')
        tu6 = self.transition_up(db6, skip_layers[1], 3)
        tu6 = Dropout(0.5)(tu6)
        
        db7 = self.dense_block(tu6, blocks[-2], name='conv7')
        tu7 = self.transition_up(db7, skip_layers[2], 3)

        db8 = self.dense_block(tu7, blocks[-3], name='conv8')
        tu8 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(db8)#(128,128,)

        uconv9 = Conv2D(512, (3, 3), activation = 'relu', padding = 'same')(tu8)
        tu9 = Conv2DTranspose(256, (3, 3), strides = (2, 2), padding = 'same')(uconv9)#(256,256,)
        tu9 = Dropout(0.3)(tu9)
        
        outputs = Conv2D(1, (1, 1), activation = 'sigmoid')(tu9)

        return outputs
    
    @tf.function
    def loss(self,y_true,y_pred):
        if tf.math.reduce_all(tf.math.equal(y_true,-1)):
            return  tf.convert_to_tensor(0, dtype=tf.float32)
        return dice_loss(y_true, y_pred)


In [ ]:
encoder = Encoder( ) 
segmenter = Segmenter(encoder)

MTL_model = ModelBlock.add_heads(encoder,[segmenter ])

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(MTL_model)

In [ ]:
#MTL_model.summary()

In [ ]:

classification_loss= "categorical_crossentropy"
detection_loss= detector.loss
segmentation_loss= segmenter.loss

mtl_loss= MTL_loss(classification_loss , detection_loss ,segmentation_loss)
lossWeights = [1.0,1.0,1.0]

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):

    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)


In [ ]:
INIT_LR = 1e-5
EPOCHS =5 

opt = Adam(lr=INIT_LR)

MTL_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[dice_coef])

In [ ]:
MTL_model.fit_generator(seg_train_gen, validation_data=seg_val_gen, epochs = 5)

In [ ]:
INIT_LR = 1e-5
EPOCHS =5 

opt = Adam(lr=INIT_LR)

MTL_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[dice_coef])

In [ ]:
MTL_model.fit_generator(seg_train_gen, validation_data=seg_val_gen, epochs = 5)

In [ ]:
MTL_model.fit_generator(seg_train_gen, validation_data=seg_val_gen, epochs = 5)